In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.layers import Activation, Flatten, Dropout
from keras.optimizers import SGD
from tqdm import tqdm_notebook
from sklearn.metrics import mean_squared_error

In [29]:
df = pd.read_csv('Sub0-RAW.csv')

In [30]:
train_cols = ["Precipitation","Temperature","Outflow"]

In [31]:
new_df = df.filter(train_cols, axis=1)

In [33]:
def timeseries(X, Y, Y_actual, time_steps, out_steps):
    input_size_0 = X.shape[0] - time_steps
    input_size_1 = X.shape[1]
    output_size_1 = Y.shape[1]
    X_values = np.zeros((input_size_0, time_steps, input_size_1))
    Y_values = np.zeros((input_size_0,output_size_1))
    Y_values_actual = np.zeros((input_size_0,output_size_1))
    
    for i in tqdm_notebook(range(input_size_0)):
        X_values[i] = X[i:time_steps+i]
        Y_values[i] = Y[time_steps+i-1]
        Y_values_actual[i] = Y_actual[time_steps+i-1]
        
    print("length of time-series i/o",X_values.shape,Y_values.shape)
    return X_values, Y_values, Y_values_actual

In [34]:
def lag_seq(df, n_seq):
    for i in range(n_seq):
        df['Precipitation(t+%d)' %(i+1)] = new_df['Precipitation'].shift(-(i+1))
        df['Temperature(t+%d)' %(i+1)] = new_df['Temperature'].shift(-(i+1))
        df['Outflow(t+%d)' %(i+1)] = new_df['Outflow'].shift(-(i+1))
    return df

In [35]:
lag_df = lag_seq(new_df, 1)
lag_df.dropna(inplace=True)

In [36]:
#label_cols = ["Outflow(t+1)", "Outflow(t+2)", "Outflow(t+3)"]
label_cols = ['Precipitation(t+1)','Temperature(t+1)','Outflow(t+1)']

In [37]:
#Min Max scalr normalizing
xtrain_min_max_scaler = MinMaxScaler(feature_range = (0, 1))
ytrain_min_max_scaler = MinMaxScaler(feature_range = (0, 1))
xtest_min_max_scaler = MinMaxScaler(feature_range = (0, 1))
ytest_min_max_scaler = MinMaxScaler(feature_range = (0, 1))

In [38]:
#Splitting training and test data
df_train, df_test = train_test_split(lag_df, train_size=0.8, test_size=0.2, shuffle=False)
x_train = df_train.loc[:,train_cols].values
y_train = df_train.loc[:,label_cols].values
x_test = df_test.loc[:,train_cols].values
y_test = df_test.loc[:,label_cols].values

In [39]:
#Normalizing training data
x_train_nor = xtrain_min_max_scaler.fit_transform(x_train)
y_train_nor = ytrain_min_max_scaler.fit_transform(y_train)

# Normalizing test data
x_test_nor = xtest_min_max_scaler.fit_transform(x_test)
y_test_nor = ytest_min_max_scaler.fit_transform(y_test)

In [40]:
y_train_actual = y_train
y_test_actual = y_test

In [41]:
#Building timeseries
X_Train, Y_Train, Y_train_actual = timeseries(x_train_nor, y_train_nor, y_train_actual, time_steps=18, out_steps=1)
X_Test, Y_Test, Y_test_actual = timeseries(x_test_nor, y_test_nor, y_test_actual, time_steps=18, out_steps=1)

/home/bidur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':



length of time-series i/o (148416, 18, 3) (148416, 3)



length of time-series i/o (37091, 18, 3) (37091, 3)


In [42]:
X_Train[1]

array([[0.        , 0.82324545, 0.00277162],
       [0.        , 0.76958782, 0.00277162],
       [0.        , 0.71611586, 0.00277162],
       [0.        , 0.6930932 , 0.00277162],
       [0.        , 0.67007055, 0.00277162],
       [0.        , 0.6470479 , 0.00277162],
       [0.        , 0.63535091, 0.00277162],
       [0.        , 0.62346825, 0.00277162],
       [0.        , 0.61158559, 0.00277162],
       [0.        , 0.6032306 , 0.00277162],
       [0.        , 0.59468994, 0.00277162],
       [0.        , 0.58633494, 0.00277162],
       [0.        , 0.58206461, 0.00277162],
       [0.        , 0.57797995, 0.00277162],
       [0.        , 0.57370962, 0.00277162],
       [0.        , 0.63943557, 0.00277162],
       [0.        , 0.70497586, 0.00277162],
       [0.        , 0.77070182, 0.00221729]])

In [44]:
Y_test_actual

array([[ 0.   , 62.348,  1.   ],
       [ 0.   , 63.212,  1.   ],
       [ 0.   , 64.076,  1.   ],
       ...,
       [ 0.   , 55.67 , 29.   ],
       [ 0.   , 57.074, 29.   ],
       [ 0.   , 57.056, 31.   ]])

In [15]:
lag_df

,Precipitation,Temperature,Outflow,Precipitation(t+1),Temperature(t+1),Outflow(t+1)
0,0.0,102.992,5.0,0.0,97.790,5.0
1,0.0,97.790,5.0,0.0,92.588,5.0
2,0.0,92.588,5.0,0.0,87.404,5.0
3,0.0,87.404,5.0,0.0,85.172,5.0
4,0.0,85.172,5.0,0.0,82.940,5.0
...,...,...,...,...,...,...
185538,0.0,52.880,29.0,0.0,54.284,30.0
185539,0.0,54.284,30.0,0.0,55.670,29.0
185540,0.0,55.670,29.0,0.0,57.074,29.0
185541,0.0,57.074,29.0,0.0,57.056,31.0


In [18]:
X_Train

array([[[0.00000000e+00, 8.76903082e-01, 2.77161863e-03],
        [0.00000000e+00, 8.23245451e-01, 2.77161863e-03],
        [0.00000000e+00, 7.69587820e-01, 2.77161863e-03],
        ...,
        [0.00000000e+00, 5.73709618e-01, 2.77161863e-03],
        [0.00000000e+00, 6.39435574e-01, 2.77161863e-03],
        [0.00000000e+00, 7.04975863e-01, 2.77161863e-03]],

       [[0.00000000e+00, 8.23245451e-01, 2.77161863e-03],
        [0.00000000e+00, 7.69587820e-01, 2.77161863e-03],
        [0.00000000e+00, 7.16115856e-01, 2.77161863e-03],
        ...,
        [0.00000000e+00, 6.39435574e-01, 2.77161863e-03],
        [0.00000000e+00, 7.04975863e-01, 2.77161863e-03],
        [0.00000000e+00, 7.70701820e-01, 2.21729490e-03]],

       [[0.00000000e+00, 7.69587820e-01, 2.77161863e-03],
        [0.00000000e+00, 7.16115856e-01, 2.77161863e-03],
        [0.00000000e+00, 6.93093205e-01, 2.77161863e-03],
        ...,
        [0.00000000e+00, 7.04975863e-01, 2.77161863e-03],
        [0.00000000e+00, 7.70

In [19]:
#Building timeseries
X_Train1, Y_Train1, Y_train_actual1 = timeseries(x_train_nor, y_train_nor, y_train_actual, time_steps=19, out_steps=1)
X_Test1, Y_Test1, Y_test_actual1 = timeseries(x_test_nor, y_test_nor, y_test_actual, time_steps=19, out_steps=1)

/home/bidur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



length of time-series i/o (148415, 19, 3) (148415,)



length of time-series i/o (37090, 19, 3) (37090,)


In [20]:
len(X_Train1)

148415

In [22]:
X_Train1[0].shape

(19, 3)

In [23]:
X_Train1[0]

array([[0.        , 0.87690308, 0.00277162],
       [0.        , 0.82324545, 0.00277162],
       [0.        , 0.76958782, 0.00277162],
       [0.        , 0.71611586, 0.00277162],
       [0.        , 0.6930932 , 0.00277162],
       [0.        , 0.67007055, 0.00277162],
       [0.        , 0.6470479 , 0.00277162],
       [0.        , 0.63535091, 0.00277162],
       [0.        , 0.62346825, 0.00277162],
       [0.        , 0.61158559, 0.00277162],
       [0.        , 0.6032306 , 0.00277162],
       [0.        , 0.59468994, 0.00277162],
       [0.        , 0.58633494, 0.00277162],
       [0.        , 0.58206461, 0.00277162],
       [0.        , 0.57797995, 0.00277162],
       [0.        , 0.57370962, 0.00277162],
       [0.        , 0.63943557, 0.00277162],
       [0.        , 0.70497586, 0.00277162],
       [0.        , 0.77070182, 0.00221729]])

In [25]:
X_Train1[0][18][2]

0.0022172949002217295

In [26]:
X_Train1[0][18][2] = 3

In [27]:
X_Train1[0]

array([[0.00000000e+00, 8.76903082e-01, 2.77161863e-03],
       [0.00000000e+00, 8.23245451e-01, 2.77161863e-03],
       [0.00000000e+00, 7.69587820e-01, 2.77161863e-03],
       [0.00000000e+00, 7.16115856e-01, 2.77161863e-03],
       [0.00000000e+00, 6.93093205e-01, 2.77161863e-03],
       [0.00000000e+00, 6.70070553e-01, 2.77161863e-03],
       [0.00000000e+00, 6.47047902e-01, 2.77161863e-03],
       [0.00000000e+00, 6.35350910e-01, 2.77161863e-03],
       [0.00000000e+00, 6.23468251e-01, 2.77161863e-03],
       [0.00000000e+00, 6.11585592e-01, 2.77161863e-03],
       [0.00000000e+00, 6.03230598e-01, 2.77161863e-03],
       [0.00000000e+00, 5.94689937e-01, 2.77161863e-03],
       [0.00000000e+00, 5.86334942e-01, 2.77161863e-03],
       [0.00000000e+00, 5.82064612e-01, 2.77161863e-03],
       [0.00000000e+00, 5.77979948e-01, 2.77161863e-03],
       [0.00000000e+00, 5.73709618e-01, 2.77161863e-03],
       [0.00000000e+00, 6.39435574e-01, 2.77161863e-03],
       [0.00000000e+00, 7.04975

In [21]:
ab = np.random.randint(1,20,(7,3))

In [22]:
ab

array([[15, 10, 10],
       [11,  8, 19],
       [ 7,  8,  8],
       [ 7, 12,  2],
       [15,  3,  4],
       [ 3,  8,  9],
       [ 5,  4, 17]])

In [25]:
np.max(ab,axis=0)

array([15, 12, 19])

In [26]:
a = np.array([-8.8274793e-04,  1.7872620e+01, -1.8418502e+00])

In [27]:
a[2]

-1.8418502